# Advanced Statistical Inference
### Raphaël Toumi's Notebook - 07/06/2019

<p style="color:blue;">
1.  Download and import the Santander dataset. The labels of the test data are not publicly available,
so create your own test set by randomly choosing half of the instances in the original training set.
</p>

In [2]:
#import necessary modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import math as mt
from numpy.linalg import inv,pinv
from sklearn.metrics import confusion_matrix

In [3]:
#Load the dataset
base = '/Users/raphaeltoumi/Documents/Cours Eurecom S1/ASI/Assessed Exercise/santander-customer-transaction-prediction/'
santanderDF = pd.read_csv(base + 'train.csv')

#Create training and test set by randomly splitting in half the original dataset
msk = np.random.rand(len(santanderDF)) < 0.5
testDF = santanderDF[~msk]
trainDF = santanderDF[msk]

<p style="color:blue;">2. Comment on the distribution of class labels and the dimensionality of the input and how these may
affect the analysis. </p>

In [4]:
#Print shape of training set
print(trainDF.shape)

(99914, 202)


The dataset we have in hand contains 200 numeric feature variables, a binary target label and an ID to identify any of the 100174 instances. Since we have a high number of features, we might want to perform some feature selection in order to reduce the dimensionality.

In [5]:
print('Ratio of instances having 1 for  label : '+str(sum(trainDF['target'])/len(trainDF)))

Ratio of instances having 1 for  label : 0.10040634946053606


Our dataset is imbalanced : there is a ratio of about 1 to 10 between the instances having 1 for label and the number of instances having 0. We will have to take this into account when evaluating the performance of our model, because even a dummy model that always output 0 will have around 90% accuracy. A better metric would be AUC (Area Under the ROC Curve), but we can also look at the confusion matrix to better explain the results.

## Bayesian Linear Regression

<p style="color:blue;">
    a) Implement Bayesian linear regression.
</p>


To implement the Bayesian linear regression, we will use a Gaussian prior $P(𝐰)=N(0,S)$ .
Because they are conjugate, the likelihood and the posterior density are also Gaussian : 

$$P(\mathbf{t}   |   \: \mathbf{w} , \: \mathbf{X},\: \sigma^2 ) =  {N}(\mathbf{X}\mathbf{w},\: \sigma^2\mathbf{I})$$

This formula tells that our observations can be expressed as a linear combination of the features added to a Gaussian noise.
Because it is conjugate, posterior must be Gaussian with unknown parameters:

$$P(\mathbf{w}   |   \: \mathbf{X} , \: \mathbf{t},\: \sigma^2 ) =  N(\: \mu,\: \Sigma)$$

Our goal is to estimate the parameters $\mu$ and $\Sigma$ of the posterior.

Ignoring constant, the posterior is:

$$P(\mathbf{w}   | \:   \mathbf{X} ,\: \mathbf{t},\: \sigma^2 ) \propto  \exp(\:-\frac{1}{2}(\mathbf{w}^T\Sigma^{-1}\mathbf{w}\:-\: 2\mathbf{w}^T\Sigma^{-1}\mu)) $$
Where the covariance matrix is: $\Sigma=(\frac{1}{\sigma^2}\mathbf{X}^T\mathbf{X}\: +\: \mathbf{S}^{-1})^{-1}$ and the mean is: $\mu=\frac{1}{\sigma^2}\Sigma\mathbf{X}^Tt$

Then we can compute the predictive density:

$$P(\mathbf{t}_{new}|\: \mathbf{X}, \: \mathbf{t}, \: \mathbf{x}_{new},\:  \sigma^2)=N(\mathbf{x}_{new}^T\mu, \: \sigma^2 \: +\:  \mathbf{x}_{new}^T\Sigma^{-1}\mathbf{x}_{new}) $$

In [6]:
def get_predictions(data, t, data_test):
    
    def weights(X,t):
        return np.array((X.T.dot(X)).I.dot(X.T).dot(t))[0]

    def variance(X, t, w):
        tmp = t - X.dot(w)
        return (tmp.dot(tmp.T)/X.shape[0])[0]

    def covariance(X, S, sigma_2):
        return inv(X.T.dot(X) / sigma_2 + inv(S))

    def mean(X, t, cov, sigma_2):
        return cov.dot(X.T).dot(t) / sigma_2
    
    X = np.matrix(data.copy())
    W = weights(X,t)
    print('weights done')
    sigma_2 = variance(X, t, W)
    print('variance done')
    S = 10*np.identity(X.shape[1])
    cov = covariance(X, S, sigma_2)
    print('covar done')
    mean =  mean(X, t, cov, sigma_2)
    print('mean done')
    # predictions
    X_test = np.matrix(data_test.copy())
    predictions = X_test.dot(mean.T)
    del X,W,S,mean
    variances = sigma_2 + X_test.dot(inv(cov)).dot(X_test.T)
    print('variances done')
    return predictions, variances

In [7]:
def get_covariance_S(X):
    prior_cov_size = X.shape[1]
    prior_cov = np.zeros((prior_cov_size,prior_cov_size)) + np.diag(np.random.uniform(0, 1, prior_cov_size))
    return (prior_cov)

def get_covariance_sigma(X, S, sigma_2):
    return np.linalg.inv(X.T.dot(X) / sigma_2 + np.linalg.inv(S))

def get_mean(X, t, covariance_sigma, variance):
    return covariance_sigma.dot(X.T).dot(t) / variance

def variance(X, t, w):
        tmp = t - X.dot(w)
        return (tmp.dot(tmp.T)/X.shape[0])[0]
    
def weights(X,t):
        return np.array((X.T.dot(X)).I.dot(X.T).dot(t))[0]

def bayesian_linear_regression(x, x_test, t, variance, degree):
    X = get_nth_order(x, degree)
    X_test = get_nth_order(x_test, degree)
    
    S = get_covariance_S(X)
    W = weights(np.matrix(X),t)
    
    covariance_sigma = get_covariance_sigma(X, S, variance)
    mu = get_mean(X, t, covariance_sigma, variance)
    
    pred_mean = X_test.dot(mu)
    pred_var = (variance + X_test.dot(np.linalg.inv(covariance_sigma)).dot(X_test.T)).diagonal()
        
    return pred_mean, pred_var

In [8]:
def bayes_lin_reg(X_train,X_test,t,order):
    # add bias to the train dataset
    X = np.append(X_train,np.ones((X_train.shape[0],1)),axis=1)

    # compute w_hat and sigma2_hat (by maximizing likelihood)
    w_hat = np.dot(np.dot(pinv(np.dot(X.T,X)),X.T),t)
    sigma2_hat = np.dot((t - np.dot(X,w_hat)).T,(t - np.dot(X,w_hat)))/X_train.shape[0]

    # add bias to test dataset
    X_new = np.append(X_test,np.ones((X_test.shape[0],1)),axis=1)
    
    # covariance of the prior
    S = 10 * np.identity(X.shape[1])
    
    # covariance of the posterior
    SIGMA = inv(1/sigma2_hat*X.T.dot(X) + inv(S))
    # mean of the posterior
    MU = 1/sigma2_hat*SIGMA.dot(X.T).dot(t)
    
    #Compute predictions and uncertainty 
    pred_mean = X_new.dot(MU)
    pred_var = (10 + X_new.dot(np.matrix(SIGMA).I).dot(X_new.T)).diagonal()
        
    return pred_mean, pred_var

<p style="color:blue;">b) Discuss how can you select the (hyper-)parameters for the Gaussian prior.</p>

We have to select two parameters for our Gaussian prior : its mean vector and its covariance matrix Σ. Since we do not know much prior knowledge, we should set its mean vector to 0 and its covariance to $𝜎^2𝐼$, with 𝐼 being the identity matrix and 𝜎 a sufficiently large positive value. We will set 𝜎=10.

<p style="color:blue;">c) Write code that calculates the N-th order polynomial transformation of the
input data. For simplicity, do not consider polynomials of more than one variable
(such as x2
y), but raise each input variable to the power of N individually.
Consider N=1, 2, 3, and 6. </p>

In [9]:
def get_nth_order(X,N):
    if N==1 :
        return np.column_stack([np.ones(X.shape[0]),X])
    else :
        X_n_1 = get_nth_order(X,N-1)
        X_add = np.empty(X.shape)
        for i in range(X.shape[0]):
            for k in range(X.shape[1]):
                X_add[i][k]=X[i][k]**N
        return np.concatenate([X_n_1,X_add],axis=1)

<p style="color:blue;">
d) Describe any additional pre-processing that you suggest for this data. </p>

Because we have a high number of features, we may want to perform feature selection, either with Principal Component Analysis (PCA) or a scoring method. We should also normalize the features before training our model to ensure numerical stability.

In [10]:
def standardized(X):
    res = np.zeros(X.shape)
    for k in range(X.shape[1]) : 
        avg = np.mean(X[:,k])
        std = np.std(X[:,k])
        res[:,k] = (X[:,k]-avg)/std
    return res

X_train = standardized(trainDF.drop(['ID_code','target'],axis=1).values)
y_train = trainDF['target'].values

X_test = standardized(testDF.drop(['ID_code','target'],axis=1).values)
y_test = testDF['target'].values

<p style="color:blue;">e) Treat class labels as continuous and apply regression to the training data.
Also, calculate the posterior variance of the weights.</p>

We will apply regression to the N-th order polynomial transformation input data for N in (1,2,3,6), and compare the results. 

In [11]:
#Try with 30% of the training set to try avoiding memory error
msk = np.random.rand(len(santanderDF)) < 0.3
trainDF_half = santanderDF[msk]

#Split 50%/50% between training and test
msk = np.random.rand(len(trainDF_half)) < 0.5
testDF_half = trainDF_half[~msk]
trainDF_half = trainDF_half[msk]

X_train = standardized(trainDF_half.drop(['ID_code','target'],axis=1).values)
y_train = trainDF_half['target'].values

X_test = standardized(testDF_half.drop(['ID_code','target'],axis=1).values)
y_test = testDF_half['target'].values
dictPerf = dict()
print(len(X_train))
for N in [1,2,3,6]:
    time0 = time.time()
    predictions, variances = bayes_lin_reg(X_train,X_test, y_train,N)
    dictPerf[N]=(predictions,variances)
    print('Time taken for computing the predictions with '+str(N)+'-th order : {:.3f} min'.format((time.time()-time0)/60))

29940
Time taken for computing the predictions with 1-th order : 1.772 min
Time taken for computing the predictions with 2-th order : 1.883 min
Time taken for computing the predictions with 3-th order : 2.001 min
Time taken for computing the predictions with 6-th order : 2.255 min


<p style="color:blue;">f) Suggest a way to discretize predictions and display the confusion matrix on the
test data and report accuracy. </p>

We can discretize our predictions by rounding to 0 if our prediction is lower than 0.5 and to 1 if our prediction is higher than 0.5.

In [12]:
def discretize(predictions):
    discretePreds = np.zeros(predictions.shape)
    for i in range(len(predictions)) :
        if predictions[i]>0.5 : 
            discretePreds[i] = 1
    return discretePreds

In [13]:
def confusion_matrix(y_true, y_pred):
    true_pos = 0
    true_neg = 0
    false_pos = 0
    false_neg = 0
    for i in range(len(y_true)):
        if y_true[i] == 0 :
            if y_pred[i] == 0:
                true_neg+=1
        else : 
            false_pos+=1
        if y_true[i] == 1 :
            if y_pred[i] == 1:
                true_pos+=1
            else : 
                false_neg+=1    
    return np.array([[true_pos, false_pos],[false_neg,true_neg]])

def accuracy(y_true,y_pred):
    m = confusion_matrix(y_true,y_pred)
    return (m[0][0]+m[1][1])/(m[0][0]+m[0][1]+m[1][0]+m[1][1])

In [14]:
for N in [1,2,3,6]:
    discretePreds = discretize(dictPerf[N][0])
    print(confusion_matrix(y_test,discretePreds))
    print('Accuracy : '+str(accuracy(y_test,discretePreds)))

[[  102  2948]
 [ 2846 26961]]
Accuracy : 0.8236601028700125
[[  102  2948]
 [ 2846 26961]]
Accuracy : 0.8236601028700125
[[  102  2948]
 [ 2846 26961]]
Accuracy : 0.8236601028700125
[[  102  2948]
 [ 2846 26961]]
Accuracy : 0.8236601028700125


<p style="color:blue;">g) Discuss the performance, compare it against a classifier that outputs random class labels.</p>

The classifiers trained with higher order polynomial transformation of the input data have exactly the same confusion matrix as the one trained with the original data. A classifier that outputs random labels would have an accuracy of around 0.5, so our classifier has a better performance. By looking at the confusion matrix, we see that our classifier predicts roughly the same ratio of positive samples than in the training data. It has a meager true positive rate of around 0.03 and a true negative rate of around 0.9. It seems that our classifier is biased toward the majority class.   

## Logistic Regression

<p style="color:blue;">
a) The goal is to implement a logistic regression classifier that optimizes for the
Maximum a Posteriori (MAP) estimate; assume a Gaussian prior on the parameters.
As a first step, write a function that calculates the gradient of the joint likelihood. </p>

We want to find the value of w that maximize the posterior probability, that is :
$$p(w|X, t, σ^2) = \frac{p(t|X, w, σ^2)p(w|σ2)}{p(t|X, σ^2)}$$

We will call this value $\hat{w}$. Since they are proportional, $\hat{w}$ also maximizes $p(t|X, w, σ^2)p(w|σ^2)$. 
Since $exp$ is an increasing function, this is equivalent to maximizing $log(p(t|X, w, σ^2)p(w|σ^2))$ .

Let's assume a centered Gaussian prior distribution of covariance $σ^2I$  : 
$p(w|σ^2) = {exp(-w^Tw/2σ)}.{(2\pi)^\frac{D}{2}}$

We also assume that the samples are independent and equally distributed. That means that the likelihood respects : 
$$p(t|X, w, σ^2) = \prod_{n=1}^N p(t_n|x_n, w))$$
where $p(t_n=1|x_n, w)=\frac{1}{1+exp(-w^Tx_n)}$ 
        and $p(t_n=0|x_n, w)=\frac{exp(-w^Tx_n)}{1+exp(-w^Tx_n)}$.
        
We can note that $$p(t_n|x_n, w)=\frac{1}{1+exp(-w^Tx_n)}^{t_n}.\frac{exp(-w^Tx_n)}{1+exp(-w^Tx_n)}^{1-t_n}$$

Let's define $h(w,X,t) = - log(𝑝(𝑡|𝑋,𝑤,σ^2)𝑝(𝑤|σ^2))$, the function we want to minimize.
To do that, we first have to compute the gradient of the log-joint-likelihood.
We will first rewrite the log-joint-likelihood :
$$log(𝑝(𝑡|𝑋,𝑤,σ^2))=\sum_{n=1}^N log(𝑝(𝑡𝑛|𝑥𝑛,𝑤))$$
$$log(𝑝(𝑡|𝑋,𝑤,σ^2))= - \sum_{n=1}^N {((1-t_n)w^Tx_n + log(1+exp(-w^Tx_n)))}$$



We can then compute its gradient :
$\frac{\partial }{\partial w}log(𝑝(𝑡|𝑋,𝑤,σ^2))=\sum_{n=1}^N {x_n(t_n - \frac{1}{1+exp(-w^Tx_n)})} $

The gradient of $h(w,X,t)$ over $w$ is then given by : $$ \frac{\partial h}{\partial w}(w,X,t) = \frac{1}{σ^2}w - \sum_{n=1}^N {x_n(t_n - \frac{1}{1+exp(-w^Tx_n)})} $$

In [39]:
sigma = 10
def gradient_log_likelihood(w,X,t):
    res=0
    for n in range(X.shape[1]):
        tmp = -w.T.dot(X[n])
        if tmp < 0 : #Distinguish those cases to avoid calling exp with a value high enough to cause overflow
            res+= (t[n]-1/(1+mt.exp(tmp)))*X[n]
        else : 
            res+= (t[n]-(1-1/(1+mt.exp(-tmp))))*X[n]
    return res

def deriv_h(w,X,t):
    return (1/sigma**2)*w - gradient_log_likelihood(w,X,t)        

<p style="color:blue;">b) Write a simple gradient descend algorithm that uses the gradients calculated
by the function of previous question to converge to the MAP estimate. </p>

In [40]:
def step(w0,X,t, min_improv, l_r,max_iter=1000):
    n_iter=0
    w_prev = w0
    w_new = w_prev - (l_r*deriv_h(w_prev,X,t))
    
    # keep looping until improvement is smaller than stopping criteria
    while sum((w_new-w_prev)**2) > min_improv and n_iter<max_iter:
        n_iter+=1
        # change the value of w
        w_prev = w_new
        
        # get the derivation of the old value of w
        d_h = deriv_h(w_prev,X,t)
        
        # get the new value of w by adding the previous, the multiplication of the derivative and the learning rate
        w_new = w_prev - (l_r * d_h)
    if n_iter == max_iter : print('Maximum iterations reached')
    return w_new

In [41]:
#Initiate w with weights chosen uniformly between 0 and 1 
w0 = np.empty(X_train.shape[1])
for k in range(X_train.shape[1]):
    w0[k] = np.random.uniform()

#Use gradient descent to find local optimum
for N in [1,2,3,6]:
    X_train_nth = get_nth_order(X_train,N)
    X_test_nth = get_nth_order(X_test,N)
    #Initiate w with weights chosen uniformly between 0 and 1 
    w0 = np.empty(X_train_nth.shape[1])
    for k in range(X_train_nth.shape[1]):
        w0[k] = np.random.uniform()
    w_hat = step(w0, X_train_nth, y_train, 0.01, 0.001, max_iter=50000)


<p style="color:blue;">c) Comment on the convexity of the problem; do you need multiple restarts in order to obtain a solution sufficiently close to the global optimum?</p>

(Show that w : h(w,X,t) is convex; Hessian? Sum of convex functions?)

Since h is convex, if h reaches a local minimum on a value $\hat{w}$, then $h(\hat{w})$ will be a global minimum. This implies that if we set a sufficiently low learning rate and let it iterate a sufficient number of times, our gradient descent will not need multiple restarts to obtain an approximate solution with any precision we would like. 

<p style="color:blue;">d) Report the confusion matrix and classification accuracy on the test data. Discuss logistic regression performance with respect to the performance of
Bayesian linear regression.</p

In [43]:
def predict(x_new,w):
    tmp = -w.T.dot(x_new)
    if tmp < 0 : #Distinguish those cases to avoid calling exp with a value high enough to cause overflow
        prob = 1/(1+mt.exp(tmp))
    else : 
        prob = 1-1/(1+mt.exp(-tmp))
    if prob > 0.5 :
        return 1
    return 0

In [44]:
y_pred = np.empty(X_test_nth.shape[0])
for n in range(X_test_nth.shape[0]):
    y_pred[n] = predict(X_test_nth[n],w_hat)
print(confusion_matrix(y_test,y_pred))
print('Accuracy : '+str(accuracy(y_test,y_pred)))

[[  617  1952]
 [ 1335 15186]]
Accuracy : 0.8278156102671556


Our classifier has a worse accuracy than the one we had with the Bayesian linear regression. It has a true positive rate of around 0.2 and a true negative rate of around 0.88. It seems that it is less biased towards the majority class than the previous classifier.

<p style="color:blue;">e) Laplace approximation is an efficient way to obtain an approximate
posterior for logistic regression. Describe the steps of this approach. What is the 
form of approximation obtained? [5] </p>

When using Laplace approximation, we want to approximate the posterior probability (that we can not compute) by a Gaussian $q(w|X, t) = N (µ, Σ)$ that is easier to work with.
The Gaussian we use to proximate the posterior has for mean vector the mode, that is the value $\hat{w}$ of which we obtained numerically an approximation earlier. Its covariance matrix verifies : $$ Σ = - \frac{\partial ^2h}{\partial {w} \partial {w^T}}log(g(w,X,t)_{|\hat{w}}$$
Given $\hat{w}$, we can compute exactly Σ.
This will give us better approximations for distributions that 'look' Gaussian.
We can then draw samples from $N (µ, Σ)$, use them to give a probability that a given instance belongs to a given class and average those probabilities to make our final prediction. 